In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=7c75fa1a77348cbe72d9250f87cc78774ce3a898c4f09b7b18c7ef52b7a37466
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=9eb39bcc6aff3fdd9b2aa3211651117bd44305a511870ee00f320f4c2d8cb20a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [54]:
fashion_mnist=keras.datasets.fashion_mnist

In [55]:
(train_images,train_lables),(test_image,test_lables)=fashion_mnist.load_data()

In [56]:
train_images=train_images/255.0
test_image=test_image/255.0


In [57]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_image = test_image.reshape(len(test_image),28,28,1)

In [58]:
def bulid_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conc_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
        activation="relu",
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conc_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values = [3,5]),
        activation="relu",
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
               loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model
              


In [59]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [60]:
tuner_search=RandomSearch(bulid_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [62]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 07m 03s]
val_accuracy: 0.8848333358764648

Best val_accuracy So Far: 0.9073333144187927
Total elapsed time: 00h 46m 28s
INFO:tensorflow:Oracle triggered exit


In [63]:
model=tuner_search.get_best_models(num_models=1)[0]

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3965056   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 4,014,570
Trainable params: 4,014,570
Non-trainable params: 0
_________________________________________________________________
